In [2]:
from dolfin import *

In [3]:
width = 1.0
height = 0.5
mesh = RectangleMesh(0, 0,width, height, 10, 2)

In [4]:
V_N = FunctionSpace(mesh, "N1curl", 1)
V_L = FunctionSpace(mesh, "CG", 1)
combined_space = V_N * V_L
(N_i, L_i) = TestFunctions(combined_space)
(N_j, L_j) = TrialFunctions(combined_space)

DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.


In [5]:
class HalfLoadedDielectric(Expression):
    def eval(self, values, x):
        if x[1] < 0.25:
            values[0] = 4.0
        else:
            values[0] = 1.0;
e_r = HalfLoadedDielectric()
one_over_u_r = Expression("1.0")
k_o_squared = Expression('0')

In [6]:
def curl_t(w):
    return Dx(w[1], 0) - Dx(w[0], 1)

In [7]:
k_o_squared = Expression("0.0")

In [14]:
s_tt = one_over_u_r*dot(curl_t(N_i), curl_t(N_j))
t_tt = e_r*dot(N_i, N_j)
s_zz = one_over_u_r*dot(grad(L_i), grad(L_j))
t_zz = e_r*L_i*L_j
b_tt = one_over_u_r*dot(N_i, N_j)*dx
b_tz = one_over_u_r*dot(N_i, grad(L_j))*dx
b_zt = one_over_u_r*dot(grad(L_i), N_j)*dx
a_tt = (s_tt - k_o_squared*t_tt)*dx
b_zz = (s_zz - k_o_squared*t_zz)*dx

In [15]:
s_tt = one_over_u_r*dot(curl_t(N_i), curl_t(N_j))*dx
t_tt = e_r*dot(N_i, N_j)*dx
s_zz = one_over_u_r*dot(grad(L_i), grad(L_j))*dx
t_zz = e_r*L_i*L_j*dx

In [23]:
B.size(0)


105

In [17]:
A = PETScMatrix()
B = PETScMatrix()

In [25]:
assemble(b_zz, tensor=B)

DEBUG:FFC:Reusing form from cache.


<dolfin.cpp.la.PETScMatrix; proxy of <Swig Object of type 'std::shared_ptr< dolfin::PETScMatrix > *' at 0x7fbada9f7300> >

In [ ]:
S = PETScMatrix()
T = PETScMatrix()
assemble(s, tensor=S)
assemble(t, tensor=T)

# Solve the eigensystem
esolver = SLEPcEigenSolver(S, T)
esolver.parameters["spectrum"] = "smallest real"
esolver.parameters["solver"] = "lapack"
esolver.solve()

In [16]:
class ElectricWalls(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

In [17]:
zero = Expression("0.0","0.0","0.0")
dirichlet_bc = DirichletBC(combined_space, zero, ElectricWalls())

TypeError: The 'element' argument must be a UFL finite element.

In [11]:
S

<dolfin.cpp.la.PETScMatrix; proxy of <Swig Object of type 'std::shared_ptr< dolfin::PETScMatrix > *' at 0x7f447ecff720> >

In [ ]:
assemble(

In [24]:
""" This demo demonstrates the calculation and visualization of a TM
    2 (Transverse Magnetic) cutoff mode of a rectangular waveguide.
    3 
    4 For more information regarding waveguides see
    5 
    6 http://www.ee.bilkent.edu.tr/~microwave/programs/magnetic/rect/info.htm
    7 
    8 See the pdf in the parent folder and the following reference
    9 
   10 The Finite Element in Electromagnetics (2nd Ed)
   11 Jianming Jin [7.2.1 - 7.2.2]
   12 
   13 """
# Copyright (C) 2008 Evan Lezar
#
# This file is part of DOLFIN.
#
# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.
#
# Modified by Anders Logg 2008, 2015
#
# First added:  2008-08-22
# Last changed: 2015-06-15

from __future__ import print_function
from dolfin import *

# Test for PETSc and SLEPc
if not has_linear_algebra_backend("PETSc"):
   print("DOLFIN has not been configured with PETSc. Exiting.")
   exit()

if not has_slepc():
   print("DOLFIN has not been configured with SLEPc. Exiting.")
   exit()

# Make sure we use the PETSc backend
parameters["linear_algebra_backend"] = "PETSc"

# Create mesh
width = 1.0
height = 0.5
mesh = RectangleMesh(0, 0,width, height, 10, 2)

# Define the function space
VN = FunctionSpace(mesh, "Nedelec 1st kind H(curl)", 3)
VL = FunctionSpace(mesh, "Lagrange", 3)
V = VN*VL
# Define the test and trial functions
v = TestFunction(V)
u = TrialFunction(V)

# Define the forms - generates an generalized eigenproblem of the form
# [S]{h} = k_o^2[T]{h}
# with the eigenvalues k_o^2 representing the square of the cutoff wavenumber
# and the corresponding right-eigenvector giving the coefficients of the
# discrete system used to obtain the approximate field anywhere in the domain
def curl_t(w):
    return Dx(w[1], 0) - Dx(w[0], 1)
s = curl_t(v)*curl_t(u)*dx
t = inner(v, u)*dx

# Assemble the stiffness matrix (S) and mass matrix (T)
S = PETScMatrix()
T = PETScMatrix()
assemble(s, tensor=S)
assemble(t, tensor=T)

# Solve the eigensystem
esolver = SLEPcEigenSolver(S, T)
esolver.parameters["spectrum"] = "smallest real"
esolver.parameters["solver"] = "lapack"
esolver.solve()

# The result should have real eigenvalues but due to rounding errors, some of
# the resultant eigenvalues may be small complex values.
# only consider the real part

# Now, the system contains a number of zero eigenvalues (near zero due to
# rounding) which are eigenvalues corresponding to the null-space of the curl
# operator and are a mathematical construct and do not represent physically
# realizable modes.  These are called spurious modes.
# So, we need to identify the smallest, non-zero eigenvalue of the system -
# which corresponds with cutoff wavenumber of the the dominant cutoff mode.
cutoff = None
for i in range(S.size(1)):
   (lr, lc) = esolver.get_eigenvalue(i)
   if lr > 1 and lc == 0:
       cutoff = sqrt(lr)
       break

if cutoff is None:
   print("Unable to find dominant mode")
else:
   print("Cutoff wavenumber:", cutoff)

DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.


Cutoff wavenumber: 7.02506101667


In [12]:
r, c, rx, cx = esolver.get_eigenpair(0)


In [13]:
plot(mesh)
interactive()

In [14]:
print(S)

<PETScMatrix of size 673 x 673>


In [ ]:
plot(u)
interactive()


In [ ]:
esolver.get_eigenpair?

In [8]:
(transverse, axial) = u.split()

In [4]:
SLEPcEigenSolver?

In [35]:
Dx?

In [36]:
inner?